In [5]:
import math
import numpy as np

from bloqade import rydberg_h, piecewise_linear, piecewise_constant, waveform, cast
from bloqade.atom_arrangement import ListOfLocations, Lieb, Square, Chain, Honeycomb, Kagome, Triangular, Rectangular
from bokeh.io import output_notebook # to plot "show()" on the notebook, without opening a browser
import os
import matplotlib.pyplot as plt

import pickle
import time

from scipy.signal import convolve2d
from pathlib import Path
import os
import re

output_notebook()

Loading BokehJS ...

In [6]:
unit_disc_grid_2_1 = {"lattice_dimension": [2, 2],"occupied_nodes": [1,2,3, 4], "unit_disc_radius": 1}
unit_disc_grid_2_sqrt2 = {"lattice_dimension": [2, 2],"occupied_nodes": [1,2,3, 4], "unit_disc_radius": np.sqrt(2)}

unit_disc_grid_3_1 = {"lattice_dimension": [3, 3],"occupied_nodes": [1,2,3, 4, 5, 6, 7, 8, 9], "unit_disc_radius": 1}
unit_disc_grid_3_sqrt2 = {"lattice_dimension": [3, 3],"occupied_nodes": [1,2,3, 4, 5, 6, 7, 8, 9], "unit_disc_radius": np.sqrt(2)}
unit_disc_grid_3_2 = {"lattice_dimension": [3, 3],"occupied_nodes": [1,2,3, 4, 5, 6, 7, 8, 9],"unit_disc_radius": 2}
unit_disc_grid_3_sqrt8 = {"lattice_dimension": [3, 3],"occupied_nodes": [1,2,3, 4, 5, 6, 7, 8, 9],"unit_disc_radius": np.sqrt(8)}


unit_disc_grid_4_sqrt2 = {"lattice_dimension": [4, 4],"occupied_nodes": [1,2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,16],"unit_disc_radius": np.sqrt(2)}     
unit_disc_grid_4_2 = {"lattice_dimension": [4, 4],"occupied_nodes": [1,2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,16],"unit_disc_radius": 2}
unit_disc_grid_4_sqrt5 = {"lattice_dimension": [4, 4],"occupied_nodes": [1,2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,16],"unit_disc_radius": np.sqrt(5)}
unit_disc_grid_4_sqrt13 = {"lattice_dimension": [4, 4],"occupied_nodes": [1,2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,16],"unit_disc_radius": np.sqrt(13)}
unit_disc_grid_4_sqrt18 = {"lattice_dimension": [4, 4],"occupied_nodes": [1,2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,16],"unit_disc_radius": np.sqrt(18)}
unit_disc_grid_4_4 = {"lattice_dimension": [4, 4],"occupied_nodes": [1,2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,16],"unit_disc_radius": 4}
unit_disc_grid_4_sqrt8 = {"lattice_dimension": [4, 4],"occupied_nodes": [1,2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,16],"unit_disc_radius": np.sqrt(8)}
unit_disc_grid_4_3 = {"lattice_dimension": [4, 4],"occupied_nodes": [1,2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,16],"unit_disc_radius": 3}


unit_disc_grid_5_sqrt5 = {"lattice_dimension": [5, 5],"occupied_nodes": [1,2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,16, 17, 18, 19, 20, 21, 22, 23, 24, 25],"unit_disc_radius": np.sqrt(5)}

unit_disc_grid_3x5_sqrt5 = {"lattice_dimension": [3, 5],"occupied_nodes": [1,2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],"unit_disc_radius": np.sqrt(5)}


In [15]:
(862690/((4*np.sqrt(5*2*np.sqrt(2)))**6))

0.07446459339133574

In [23]:
def norm(x):
    return sum([ i**2 for i in x])

def upper_bound(r):
    L = [ (i,j) if norm((i,j)) > r**2 else [2*r,0] for i in range(math.ceil(r), math.ceil(r)+4) for j in range(i+1)]
    return min(map(lambda x: math.sqrt(norm(x)), L))


def lower_bound(r):
    L = [ (i,j) if norm((i,j))<= r**2 else [0,0] for i in range(math.floor(r)+2) for j in range(i+2) ]
    R  = max(map(lambda x: math.sqrt(norm(x)) , L))
    return R
    
def radius_interval(r):
    return [lower_bound(r), upper_bound(r)]


def circle_kernel(radius):
    kernel_size = int(np.floor(radius))*2+1
    kernel = np.zeros((kernel_size, kernel_size))
    
    for i in range(kernel_size):
        for j in range(kernel_size):
            if (radius)**2 >= (i-int(radius))**2+(j-int(radius))**2:
                kernel[i][j] = 1
    kernel[int(radius)][int(radius)] = 0
    return kernel


def is_independent(arr, kernel, radius, return_missed=True):
    kernel_sum = np.sum(kernel)
    padded = np.pad(arr, int(radius), constant_values=1)
    convolved = convolve2d(padded, kernel, mode="valid")

    # print(arr)
    # print(convolved)
    # print((convolved[np.bitwise_not(arr.astype(bool))] == kernel_sum).astype(int))
    # print()
    is_independent = np.all(convolved[np.bitwise_not(arr.astype(bool))] == kernel_sum)

    if return_missed:
        return is_independent, ((convolved == kernel_sum) ^ arr).astype(bool)
    else: 
        return is_independent


def peform_sweep(unit_disc, opt_steps, save_name):
    os.makedirs(save_name, exist_ok=True)

    index = 1
    location_list = []
    for x in range(unit_disc["lattice_dimension"][0]):
        for y in range(unit_disc["lattice_dimension"][1]):
            if index in unit_disc["occupied_nodes"]:
                location_list.append((x*4, y*4))
            index = index + 1
    
    atom_coordinate = ListOfLocations(location_list)
    
    
    #output = program.bloqade.python().run(shots=100, interaction_picture=True)
    
    #dynamics
    durations = [0.2,3.6,0.2]
    r_min, r_max = radius_interval(unit_disc["unit_disc_radius"])
    r_min = np.sqrt(5)
    r_max = np.sqrt(2)*2
    d_end = (862690/((4*r_min)**6))
    d_start = (862690/((4*r_max)**6))

    t = time.localtime()

    #print(np.geomspace(d_start, d_end, 8))
    #print([((862690/i)**(1/6)/4) for i in np.geomspace(d_start, d_end, 8)])
    for i, d_max in enumerate(np.geomspace(d_start, d_end, 8)):
        
        
        delta_MHz= [-11,-11, d_max,d_max]
        
        Om_max = 2.5
        #if (d_max/Om_max <1.5):
            #Om_max = d_max/1.5

        omega_MHz= [0,Om_max, Om_max,0] 
        
        Delta = piecewise_linear(durations,[x*2*np.pi for x in delta_MHz])
        Omega = piecewise_linear(durations,[x*2*np.pi for x in omega_MHz])
    
        #create Hamiltonian
        program = rydberg_h(atom_coordinate, detuning= Delta, amplitude=Omega, phase=None)

        program.parse_register()
        program.parse_sequence()
        
        output = program.bloqade.python().run(shots=100, interaction_picture=True)
        output.report().show()

        with open(f'{save_name}/step={i}.pickle', 'wb') as file:
             pickle.dump({
                "d_max": d_max,
                "bitstrings": output.report().bitstrings(),
                "counts": output.report().counts(),
                "densities": output.report().rydberg_densities(),
                "dataframe": output.report().dataframe,
                "unit_disk": unit_disc,
                "R": (r_min, r_max),
                 "r/a": ((862690/d_max)**(1/6))/4
            }, file)
            
        

In [20]:

def independent_set_probabilities(save_path):
    data = {}
    for pickle_file in list(Path(f"{save_path}/").glob('*.pickle')):
        with pickle_file.open('rb') as file: sweep_data = pickle.load(file)
        step = int(''.join(re.findall(r'\d', str(pickle_file))))
        data[step]= {}
        
        radius = sweep_data["unit_disk"]["unit_disc_radius"]
        kernel = circle_kernel(radius)
        x_size, y_size = sweep_data["unit_disk"]["lattice_dimension"]
        arrays = sweep_data["bitstrings"][0]

        unique, counts = np.unique(arrays, axis=0, return_counts=True)
        
        indep_sets_idx = []
        cardininalities = []
        for i, array in list(enumerate(unique)):
            if is_independent(array.reshape(x_size, y_size), kernel, radius, return_missed=False):
                indep_sets_idx.append(i)
                cardininalities.append(array.size - np.sum(array))

        indep_sets_idx = np.array(indep_sets_idx)[np.argsort(cardininalities)]
        cardininalities = np.array(np.sort(cardininalities))

        prob = []
        for c in np.unique(cardininalities):
            fixed_card = indep_sets_idx[np.where(cardininalities==c)]
            prob.append(np.sum(counts[fixed_card])/np.sum(counts))

        data[step]["prob"] = prob[::-1]
        data[step]["cardinality"] = np.unique(cardininalities)[::-1]

    with open(f'{save_path}/indep_set_probabilities.pickle', 'wb') as file:
            pickle.dump(data, file)     

# independent_set_probabilities("test")
    

In [13]:
peform_sweep(unit_disc_grid_2_sqrt2, 4, "test")

In [24]:
peform_sweep(unit_disc_grid_3x5_sqrt5, 4, "run3_grid_3x5_sqrt5")